# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [1]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np

In [2]:
dvach = open('data/2ch_corpus.txt', encoding='utf-8').read()
news = open('data/lenta.txt', encoding='utf-8').read()

In [3]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [4]:
from collections import Counter

In [5]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0, len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [6]:
sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [7]:
sentences = []
sentences.extend(sentences_dvach)
sentences.extend(sentences_news)
sentences = [s.split() for s in set([' '.join(s) for s in sentences])]

In [8]:
import random

In [9]:
random.shuffle(sentences)

In [10]:
max_sample = len(sentences)
train_sentences = sentences[:max_sample - int(max_sample*0.2) - 1]
test_sentences = random.sample(sentences[len(train_sentences) + 1:], 20)

In [11]:
unigrams = Counter()
bigrams  = Counter()
trigrams = Counter()

for sentence in train_sentences:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence))
    trigrams.update(ngrammer(sentence, n=3))

In [12]:
from scipy.sparse import lil_matrix

In [13]:
all_matrix = lil_matrix((len(bigrams), 
                         len(unigrams)),)
rows = list(bigrams)
rows_id = {bigram:i for i, bigram in enumerate(rows)}
cols = list(unigrams)
cols_id = {word:i for i, word in enumerate(cols)}

for ngram in trigrams:
    word1, word2, word3 = ngram.split()
    all_matrix[rows_id[word1 + ' ' + word2], cols_id[word3]] =  (trigrams[ngram]/
                                                                     bigrams[word1 + ' ' + word2])

In [14]:
def generate(matrix, cols, rows, rows_id, n=100, start='<start> <start>'):
    text = []
    current_idx = rows_id[start]
    for i in range(n):
        chosen = np.random.choice(matrix.getrow(current_idx).toarray().shape[1], p=matrix.getrow(current_idx).toarray()[0])
        text.append(cols[chosen])
        if cols[chosen] == '<end>':
            current_idx = rows_id[start]
        else:
            phrase = rows[current_idx] + ' ' + cols[chosen]
            phrase = ' '.join(phrase.split()[1:])
            current_idx = rows_id[phrase]
    return ' '.join(text)

In [21]:
print(generate(all_matrix, cols, rows, rows_id).replace('<end>', '\n'))

чувак ты лучший 
 часами напролет пиздели до рассвета среды 
 в ходе которых он объявил жителям республики так и показал чему учился долгие годы считалась утраченной принадлежит родственнице покойного писателя василия кудашова друга михаила шолохова тихий дон найденная у частного лица будет приобретена другим известным производителем сигарет gauloises и gitanes крупнейшей табачной компанией франции 
 почему у австралии совершенно не знаю какую то подставу 
 я лично к саудитам ездил совещаться по этому делу задержаны 22 обвиняемых 
 ты не понимаешь не о исламе ни о чём так приятно слушать потому что когда подписывался « минск-2 » дебальцевский район


Результаты генерации получаются не совсем осмысленные, но иногда получаются забавные предложения, которые соединяются с двачом и новостями. 

In [16]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N)

In [17]:
perplexities = []
for sentence in test_sentences:
    prob = []
    
    for ngram in ngrammer(sentence, n=3):
        word1, word2, word3 = ngram.split()

        www = word1 + ' ' + word2
        if www in bigrams and ngram in trigrams:
            prob.append(np.log(trigrams[ngram]/bigrams[word1 + ' ' + word2]))
        else:
            prob.append(np.log(0.00001))
    
    perplexities.append(perplexity(prob))
    perplexities.append(perplexity(prob))
    print(f'sentence: {" ".join(sentence)}')
    print(f'peplexity: {perplexities[-1]}')

sentence: <start> <start> сложно оценивать <end>
peplexity: 57399.977337743534
sentence: <start> <start> каждая глава законченная мысль <end>
peplexity: 63037.11262727307
sentence: <start> <start> невынужденных ошибок что впрочем никак не помогло его сопернику <end>
peplexity: 54861.75487023924
sentence: <start> <start> это очень легко <end>
peplexity: 44.37335394545081
sentence: <start> <start> твой контролер просто изначально может выдавать 127 сюрприз но многие могут максимум 100-110 <end>
peplexity: 80134.6638451409
sentence: <start> <start> именно поэтому они отступили сконцентрировавшись на охране периметра и оставив всякую надежду управлять внутренним распорядком подопечных <end>
peplexity: 43973.88657598156
sentence: <start> <start> впервые об этом открыто сказал на прошлой неделе директор цру джордж тенет <end>
peplexity: 3215.600324593135
sentence: <start> <start> российский партнер телекомпании bloomberg канал нтв-плюс входит в холдинг медиа-мост и является первой в россии н

In [18]:
print(f'mean perplexity: {np.mean(perplexities)}')

mean perplexity: 29643.698215166678


Можно увидеть, что перплексия на корпусе из двача и новостей показывает в среднем не очень хорошие результаты. Чтобы повысить качество, возможно нужно добавить ещё больше данных.

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

Статистические модели (n-граммная модель является таковой) очень зависят от данных, потому может быть такая ситуация, когда некоторые слова могут не оказаться в тестовой выборке. Такие слова называются неизвестными или выходящие из словаря (out of vocabulary (OOV)). OOV rate называют процент неизвестных слов, которые встретились в тестовой выборке. Чтобы решить такую проблему авторы книги предлагают два пути решения, но главная их идея состоит в присвоении псевдослова "< UNK >" неизвестным словам.

Первый подход состоит в том, что создаётся фиксированный список слов, в тренировочной выборке слова, которые не попадают в этот список, преобразуются в псевдослово "< UNK >" и, наконец, считается вероятность этих слов, как для других.

Второй подход заключается в том, чтобы заменять те слова, у которых количество появлений в тренировочном наборе меньше определенного числа, на псевдослово.

2. Что такое сглаживание (smoothing)?

Под сглаживанием подразумевается процесс, когда для слов, которые находятся в тренировочном наборе, но контекст для них отсутствует в тестовом, вместо зануления добавляется какая-то часть вероятности, которая берётся у слов c наиболее частотными событиями.